In [2]:
import os
import importlib , pck, pandas as pd # Importar importlib para recargar módulos
from dotenv import load_dotenv

dotenv_path = os.path.join(os.getcwd(), '..', '.env')
importlib.reload(pck)
from pck import PCK

# Configuracion
config_file = os.path.join(os.getenv('BASE_PATH'), os.getenv('MMPOSE_SUBPATH'), os.getenv('CONFIG_SUBPATH'))
checkpoint_file = os.path.join(os.getenv('BASE_PATH'), os.getenv('MMPOSE_SUBPATH'), os.getenv('CHECKPOINT_SUBPATH'))
images_path = os.path.join(os.getenv('BASE_PATH'), os.getenv('IMAGES_SUBPATH'))
labels_path = os.path.join(os.getenv('BASE_PATH'), os.getenv('LABELS_SUBPATH'))
evaluator = PCK(os.getenv('BASE_PATH'), images_path, labels_path, config_file, checkpoint_file)
thresholds = [0.1, 0.5, 0.9]

Loads checkpoint by local backend from path: /Users/apple/Documents/pose_estimation/MMPosev1.1/configs/model_weights/hrnet_w32_coco_256x192-c78dce93_20200708.pth


/Users/apple/Documents/pose_estimation/MMPosev1.1/mmpose_env/lib/python3.10/site-packages/mmpose/datasets/datasets/utils.py:102: UserWarning: The metainfo config file "configs/_base_/datasets/coco.py" does not exist. A matched config file "/Users/apple/Documents/pose_estimation/MMPosev1.1/mmpose_env/lib/python3.10/site-packages/mmpose/.mim/configs/_base_/datasets/coco.py" will be used instead.
  warnings.warn(


# Evaluando una imagen

In [ ]:
image = 'imagen_062_jpg.rf.67b212e193dcccad590e8a4eb2dc24cc.jpg'
results = []
for threshold in thresholds:
    results = evaluator.evaluate_image(image, threshold, results)

# Infernecia de una imagen

In [ ]:
import re
import matplotlib.pyplot as plt
fig, axes = plt.subplots(1, 2, figsize=(6, 3)) 
image_path = os.path.join(images_path, image)
label_path = os.path.join(labels_path, os.path.splitext(image)[0] + '.txt')

img_with_original_keypoints = evaluator.draw_original_keypoints(image_path, label_path)
img_with_predicted_keypoints, inference_results = evaluator.draw_predicted_keypoints(image_path)

# extraer todo lo que esta antes de '.rf'
match = re.match(r"^(.*?)(_jpg)\.rf\.", image)

image_name = match.group(1) + match.group(2) if match else 'Not found'

# Mostrar la imagen con etiquetas originales
axes[0].imshow(img_with_original_keypoints)
axes[0].set_title(f'{image_name}')
axes[0].axis('off')

# Mostrar la imagen con etiquetas predichas
axes[1].imshow(img_with_predicted_keypoints)
axes[1].set_title(f'Prediccion - MPJPE 13.78')
axes[1].axis('off')

# Ajustar el espacio entre subplots
plt.tight_layout()

# Mostrar la figura
plt.show()

# Evaluando multiples imagenes

In [4]:
# Obtener la lista de todas las imágenes en la carpeta
image_files = [f for f in os.listdir(images_path) if f.endswith('.jpg')]
image_files_sorted = sorted(image_files)
results = []

for image in image_files_sorted:
    for threshold in thresholds:
        results = evaluator.evaluate_image(image, threshold, results)

PCK: 100.00%, Total true: 17, Visible true: 12, Total pred: 17, Visible pred: 12
PCK: 100.00%, Total true: 17, Visible true: 12, Total pred: 17, Visible pred: 12
PCK: 100.00%, Total true: 17, Visible true: 12, Total pred: 17, Visible pred: 12
PCK: 100.00%, Total true: 17, Visible true: 11, Total pred: 17, Visible pred: 11
PCK: 100.00%, Total true: 17, Visible true: 11, Total pred: 17, Visible pred: 11
PCK: 100.00%, Total true: 17, Visible true: 11, Total pred: 17, Visible pred: 11
PCK: 100.00%, Total true: 17, Visible true: 17, Total pred: 17, Visible pred: 17
PCK: 100.00%, Total true: 17, Visible true: 17, Total pred: 17, Visible pred: 17
PCK: 100.00%, Total true: 17, Visible true: 17, Total pred: 17, Visible pred: 17
PCK: 100.00%, Total true: 17, Visible true: 13, Total pred: 17, Visible pred: 13
PCK: 100.00%, Total true: 17, Visible true: 13, Total pred: 17, Visible pred: 13
PCK: 100.00%, Total true: 17, Visible true: 13, Total pred: 17, Visible pred: 13
PCK: 100.00%, Total true: 17

# Generando el XLS

In [6]:
# Crear un DataFrame con los resultados
df_results = pd.DataFrame(results)
output_excel_path = os.path.join(os.getenv('BASE_PATH'), os.getenv('MMPOSE_SUBPATH'), 'precision', 'PCK', 'mmpose_pck_results.xlsx')
df_results.to_excel(output_excel_path, index=False, engine='openpyxl')

print(f"Los resultados se han guardado en {output_excel_path}")

Los resultados se han guardado en /Users/apple/Documents/pose_estimation/MMPosev1.1/precision/PCK/mmpose_pck_results.xlsx
